# Performance Report

In [ ]:
%load_ext autoreload
%autoreload 2

## Imports

In [ ]:
from deepchecks.vision.base import VisionDataset
from torchvision import models
import torchvision
import torch
from torch import nn
import torch.nn.functional as F
from torchvision.transforms import ToTensor
import copy

In [ ]:
from deepchecks.vision.checks.performance import PerformanceReport

## Loading data and model:

In [ ]:
from deepchecks.vision.datasets.classification import mnist

In [ ]:
def simple_nn():
    torch.manual_seed(42)

    # Define model
    class NeuralNetwork(nn.Module):
        def __init__(self):
            super(NeuralNetwork, self).__init__()
            self.flatten = nn.Flatten()
            self.linear_relu_stack = nn.Sequential(
                nn.Linear(28 * 28, 512),
                nn.ReLU(),
                nn.Linear(512, 512),
                nn.ReLU(),
                nn.Linear(512, 10)
            )

        def forward(self, x):
            x = self.flatten(x)
            logits = self.linear_relu_stack(x)
            return logits

    model = NeuralNetwork().to('cpu')
    return model

def trained_mnist(simple_nn, mnist_data_loader_train):
    torch.manual_seed(42)
    simple_nn = copy.deepcopy(simple_nn)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(simple_nn.parameters(), lr=1e-3)
    size = len(mnist_data_loader_train.dataset)
    # Training 1 epoch
    simple_nn.train()
    for batch, (X, y) in enumerate(mnist_data_loader_train):
        X, y = X.to('cpu'), y.to('cpu')

        # Compute prediction error
        pred = simple_nn(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    return simple_nn

In [ ]:
train_ds = mnist.load_dataset(train=True, object_type='Dataset')
test_ds = mnist.load_dataset(train=False, object_type='Dataset')

In [ ]:
model = trained_mnist(simple_nn(), train_ds.get_data_loader())

## Running peformance report on classification

In [ ]:
check = PerformanceReport(prediction_extract=nn.Softmax(dim=1))
check.run(train_ds, test_ds, model)

# Performance report object detection

In [ ]:
from deepchecks.vision.datasets.detection import coco
import numpy as np

In [ ]:
yolo = coco.load_model(pretrained=True)

In [ ]:
train_ds = coco.load_dataset(train=True, object_type='Dataset', batch_size=64)
test_ds = coco.load_dataset(train=False, object_type='Dataset', batch_size=64)

In [ ]:
check = PerformanceReport(prediction_extract=coco.yolo_wrapper)
check.run(train_ds, test_ds, yolo)